# Diabetes prediction from UCI diabetes data

In [1]:
# Importing all used libraries
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier

## Introduction
Diabetes mellitus, commonly referred to as diabetes is a disease which impacts the body’s control of blood glucose levels (Sapra, Bhandari 2023). It is important to note that there are different types of diabetes, although we do not explore this discrepancy in this project (Sapra, Bhandari 2023). Diabetes is a manageable disease thanks to the discovery of insulin in 1922. Globally, 1 in 11 adults have diabetes (Sapra, Bhandari 2023). As such, understanding the factors which are strongly related to diabetes can be important for researchers studying how to better prevent or manage the disease. In this project, we create several machine learning models to predict diabetes in a patient and evaluate the success of these models. We also explore the coefficients of a logistic regression model to better understand the factors which are associated with diabetes. 

In [2]:
diabetes_df = pd.read_csv("data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv")
diabetes_df.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [3]:
diabetes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [4]:
print(diabetes_df.shape)
diabetes_df.describe().T


(70692, 22)


,count,mean,std,min,25%,50%,75%,max
Diabetes_binary,70692.0,0.500000,0.500004,0.0,0.0,0.5,1.0,1.0
HighBP,70692.0,0.563458,0.495960,0.0,0.0,1.0,1.0,1.0
HighChol,70692.0,0.525703,0.499342,0.0,0.0,1.0,1.0,1.0
CholCheck,70692.0,0.975259,0.155336,0.0,1.0,1.0,1.0,1.0
BMI,70692.0,29.856985,7.113954,12.0,25.0,29.0,33.0,98.0
Smoker,70692.0,0.475273,0.499392,0.0,0.0,0.0,1.0,1.0
Stroke,70692.0,0.062171,0.241468,0.0,0.0,0.0,0.0,1.0
HeartDiseaseorAttack,70692.0,0.147810,0.354914,0.0,0.0,0.0,0.0,1.0
PhysActivity,70692.0,0.703036,0.456924,0.0,0.0,1.0,1.0,1.0
Fruits,70692.0,0.611795,0.487345,0.0,0.0,1.0,1.0,1.0


In [5]:
# Check for duplicate in dataset
duplicate_rows = diabetes_df.duplicated()
print(duplicate_rows.value_counts())

False    69057
True      1635
Name: count, dtype: int64


In [7]:
# Check for imbalance dataset
diabetes_df.drop_duplicates(inplace=True)
diabetes_df["Diabetes_binary"].value_counts()

Diabetes_binary
1.0    35097
0.0    33960
Name: count, dtype: int64

In [8]:
# Check for null values
diabetes_df.isnull().sum()

Diabetes_binary         0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

In [3]:
#Creating train and test data
train_df, test_df = train_test_split(diabetes_df, test_size = 0.2, random_state=123)

X_train = train_df.drop(columns = "Diabetes_binary")
y_train = train_df["Diabetes_binary"]

X_test = test_df.drop(columns = "Diabetes_binary")
y_test = test_df["Diabetes_binary"]

In [ ]:
# plotting histogram distributions
alt.data_transformers.enable("vegafusion")
numeric_cols = train_df.select_dtypes(include=['float64']).columns.to_list()

hist_plot = alt.Chart(train_df).mark_bar(opacity=0.7).encode(
            x=alt.X(alt.repeat(),type='quantitative', bin=alt.Bin(maxbins=20)),
            y=alt.Y('count()').stack(False),
            color=alt.Color('Diabetes_binary:N')
        ).properties(
            width=150,
            height=150
        ).repeat(
            numeric_cols,
            columns=4
        )

hist_plot

In [25]:
#Creating the baseline for our model
dummy = DummyClassifier()
scores = cross_validate(dummy, X_train, y_train, return_train_score=True)
pd.DataFrame(scores)

,fit_time,score_time,test_score,train_score
0,0.004000,0.001001,0.510363,0.510408
1,0.005999,0.002000,0.510363,0.510408
2,0.004999,0.001001,0.510363,0.510408
3,0.003998,0.001001,0.510453,0.510386
4,0.004001,0.000999,0.510453,0.510386


# Model comparison

In [4]:
# Designate binary and continuous cols
binary_cols = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 
               'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
              'DiffWalk', 'Sex']
continuous_cols = ['BMI', 'Age', 'GenHlth', 'MentHlth', 'PhysHlth', 'Education', 'Income']


In [5]:
# Create a pre-processor which scales the continuous cols
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', StandardScaler(), continuous_cols),
        ('binary', 'passthrough', binary_cols)
    ])

In [6]:
# Models to test
models = {
    "Dummy": make_pipeline(preprocessor, DummyClassifier()),
    "Decision tree": make_pipeline(preprocessor, DecisionTreeClassifier(random_state=123)),
    "Logistic regression": make_pipeline(preprocessor, LogisticRegression()),
    "Knn": make_pipeline(preprocessor, KNeighborsClassifier())
}

In [7]:
# Evaluate each model
results_dict = {}

for name, pipeline in models.items():
    
    # Cross-validation on training data
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv = 5)
    mean_cv_score = round(cv_scores.mean(), 2)

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    test_accuracy = round(accuracy_score(y_test, predictions), 2)

    results_dict[name] = (mean_cv_score, test_accuracy)

results_df = pd.DataFrame(list(results_dict.values()), index=results_dict.keys(), columns=['Mean CV Score', 'Test Accuracy'])
results_df

,Mean CV Score,Test Accuracy
Dummy,0.50,0.50
Decision tree,0.66,0.66
Logistic regression,0.75,0.75
Knn,0.71,0.71


# Feature Importance

In [8]:
# Manually scaling the data
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train_scaled = scaler.transform(X_train)  
X_test_scaled = scaler.transform(X_test)

In [10]:
# Show coefficients
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
cols = train_df.drop(columns=["Diabetes_binary"]).columns
data = {"features": cols, "coefficients": lr.coef_[0]}
pd.DataFrame(data)

,features,coefficients
0,HighBP,0.367749
1,HighChol,0.291999
2,CholCheck,0.219338
3,BMI,0.535329
4,Smoker,0.005788
5,Stroke,0.031232
6,HeartDiseaseorAttack,0.095081
7,PhysActivity,-0.012959
8,Fruits,-0.016229
9,Veggies,-0.028698
